# Model Training

### Importing packages

In [2]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten, Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='4H3FV4X0EBpzb0UEcZRA2IeCYKPP5z9mYcmf0IA73kPD',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'gesturebrowsing-donotdelete-pr-slzzfpa7bpxsk1'
object_key = 'Dataset.zip'

streaming_body_4 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_4.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

### Image Data Argumentation

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
pwd

'/home/wsuser/work'

### Loading Our Data And Perform Data Argumentation

In [7]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/train', target_size=(64, 64), batch_size=3, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('/home/wsuser/work/Dataset/test', target_size=(64, 64), batch_size=3, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [8]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='4H3FV4X0EBpzb0UEcZRA2IeCYKPP5z9mYcmf0IA73kPD',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'gesturebrowsing-donotdelete-pr-slzzfpa7bpxsk1'
object_key = '8.jpg'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
print(x_train.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


### Initializing The Model

In [9]:
model=Sequential()

### Adding CNN Layers

In [10]:
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [11]:
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
model.add(Flatten())

### Adding Dense Layers

In [13]:
model.add(Dense(units=512, activation='relu'))

In [14]:
model.add(Dense(units=6, activation='softmax'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 512)               3

### Configure The Learning Process

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train The Model

In [17]:
model.fit(x_train,
                    steps_per_epoch =594/3,
                    epochs=25,
                    validation_data=x_test,
                    validation_steps=30/3)

Epoch 1/25
198/198 [==============================] - 6s 28ms/step - loss: 1.3003 - accuracy: 0.4983 - val_loss: 0.9333 - val_accuracy: 0.6333
Epoch 2/25
198/198 [==============================] - 5s 26ms/step - loss: 0.5841 - accuracy: 0.7677 - val_loss: 0.3185 - val_accuracy: 0.9000
Epoch 3/25
198/198 [==============================] - 5s 27ms/step - loss: 0.3695 - accuracy: 0.8687 - val_loss: 0.2619 - val_accuracy: 0.9333
Epoch 4/25
198/198 [==============================] - 5s 27ms/step - loss: 0.2775 - accuracy: 0.9040 - val_loss: 0.2018 - val_accuracy: 0.9000
Epoch 5/25
198/198 [==============================] - 5s 27ms/step - loss: 0.2552 - accuracy: 0.9074 - val_loss: 0.1923 - val_accuracy: 0.9667
Epoch 6/25
198/198 [==============================] - 5s 27ms/step - loss: 0.1481 - accuracy: 0.9444 - val_loss: 0.2105 - val_accuracy: 0.9000
Epoch 7/25
198/198 [==============================] - 5s 27ms/step - loss: 0.1696 - accuracy: 0.9428 - val_loss: 0.2141 - val_accuracy: 0.9000

### Save The Model

In [18]:
model.save('gesture.h5')

In [19]:
!tar -zcvf gesture.tgz gesture.h5

gesture.h5


In [20]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 17.5 MB/s eta 0:00:01


In [21]:
from ibm_watson_machine_learning import APIClient

API_KEY = "QkvIoyc0w4W0D8DCvZ4HSwW5xvwAxW-uqhTQZM6QhwX_"

wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(wml_credentials)

In [22]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [23]:
space_uid = guid_from_space_name(client, 'A Gesture-based Images')
print("Space UID: ", space_uid)

StopIteration: 